In [12]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os
import imutils

In [13]:
from pygame import mixer

class SoundAlert:

    mixer.init()                              # intitializing our mixer for beep

    def sound_alert(self):
        sound = mixer.Sound(r'C:\Users\User\Downloads\bb.wav')         # passed our required audio sound beep
        sound.play()

In [14]:
ss=SoundAlert()
#ss.sound_alert()

In [15]:
prototxtPath = r"C:\Users\User\Downloads\Face mask detection files\deploy.prototxt.txt"
weightsPath =  r"C:\Users\User\Downloads\Face mask detection files\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [16]:
# load the face mask detector model from disk
maskNet = load_model("mask_detector.model")

In [17]:
frames = r"C:\Users\User\Downloads\37.jpg"
frame=cv2.imread(frames)
frame = imutils.resize(frame, width=200)
(h, w) = frame.shape[:2]
blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))
    
# pass the blob through the network and obtain the face detections
faceNet.setInput(blob)
detections = faceNet.forward()
print(detections.shape)

(1, 1, 101, 7)


In [ ]:
withMaskColor, withoutMaskColor  = (0, 255, 0), (0, 0, 255)
font = cv2.FONT_HERSHEY_SIMPLEX


for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    
    if confidence > 0.5:
        #bounding box
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        
        #ensure bounding box within dimensions of frame
        (startX, startY)= (max(0, startX), max(0, startY))
        (endX, endY) = (min(w-1, endX), min(h-1, endY))
        
        #process to feed into model
        face = frame[startY: endY, startX: endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (244,244))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)
        
        (mask, withoutMask) = maskNet.predict(face)[0]
        
        # display result
        
        label = "Mask" if mask> withoutMask else "No Mask"
        sound = ss.sound_alert() if label == "No Mask" else "mic"
        color = withMaskColor if label=="Mask" else withoutMaskColor

        #adding accouricy
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        #display label along with rectangle
        cv2.putText(frame, label, (startX, startY-10), font, .5, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        
cv2.imshow("outPut", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()